In [1]:
# Imports and useful definitions
import random
import numpy as np
import pandas as pd
from IPython.display import display
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

random.seed(156)
pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [2]:
# Load train data
df_train = pd.read_csv("raw_data/train.csv",
                       dtype = {'Semana' : 'int8', 'Agencia_ID' :'int16', 'Canal_ID' : 'int8', 'Ruta_SAK' : 'int16',
                                'Cliente-ID' : 'int32', 'Producto_ID':'int32', 'Venta_uni_hoy':'int16', 'Venta_hoy': 'float32', 
                                'Dev_uni_proxima':'int32', 'Dev_proxima':'float32', 'Demanda_uni_equil':'int16'})
df_train.columns = ["week", "depot_id", "channel_id", "route_id", "store_id", "product_id", "sales_nb", "sales_pesos", 
                    "returns_nb", "returns_pesos", "adj_demand"]

In [3]:
# First preprocess Products data
df_products = pd.read_csv("raw_data/producto_tabla.csv")
df_products.columns = ["product_id", "product_name"]

# Augment the product DF using the data from product_name
df_products["short_name"] = df_products.product_name.str.extract('^(\D*)', expand = False)
df_products["pieces"] =  df_products.product_name.str.extract('(\d+)p ', expand = False).astype('float')
weight = df_products.product_name.str.extract('(\d+)(kg|Kg|g) ', expand = True)
df_products["weight"] = weight[0].astype('float') * weight[1].map({'Kg':1000, 'kg':1000, 'g':1})
df_products["volume"] =  df_products.product_name.str.extract('(\d+)ml ', expand = False).astype('float')
df_products["inches"] =  df_products.product_name.str.extract('(\d+)in ', expand = False).astype('float')
df_products["brand"] = df_products.product_name.str.extract('^.+\s(\D+) \d+$', expand = False)

# TODO : handle digits in product name "Tostado Int 0pct Grasa Azuc" and packaging "NM MTA"

# Remove NO IDENTIFICADO entries
df_products = df_products[df_products.short_name != "NO IDENTIFICADO "]

# Infer several features like average price from trains.csv
df_train["product_price"] = df_train.sales_pesos / df_train.sales_nb
avg_prices = df_train.groupby("product_id")["product_price"].mean().to_frame()
df_products = pd.merge(left = df_products, right = avg_prices, how = "left", left_on = "product_id", right_index = True)
df_products.rename(columns = {"product_price" : "avg_price"}, inplace = True)

total_sales_nb = df_train.groupby("product_id")["sales_nb"].sum().to_frame()
df_products = pd.merge(left = df_products, right = total_sales_nb, how = "left", left_on = "product_id", right_index = True)
df_products.rename(columns = {"sales_nb" : "total_sales_nb"}, inplace = True)

total_return_nbs = df_train.groupby("product_id")["returns_nb"].sum().to_frame()
df_products = pd.merge(left = df_products, right = total_return_nbs, how = "left", left_on = "product_id", right_index = True)
df_products.rename(columns = {"returns_nb" : "total_returns_nb"}, inplace = True)

total_adj_demand = df_train.groupby("product_id")["adj_demand"].sum().to_frame()
df_products = pd.merge(left = df_products, right = total_adj_demand, how = "left", left_on = "product_id", right_index = True)
df_products.rename(columns = {"adj_demand" : "total_adj_demand"}, inplace = True)

# Remove rows with products who were never sold or returned once (thus providing no information)
df_products.dropna(axis = 0, how = "any", subset  = ["total_adj_demand"], inplace = True)

# Use product_id as the index
df_products.set_index("product_id", inplace = True)

display("Dimensions of Products table are now : " + str(df_products.shape))

'Dimensions of Products table are now : (1799, 11)'

In [4]:
# Now preprocess Stores data
df_stores = pd.read_csv("raw_data/cliente_tabla.csv")
df_stores.columns = ["store_id", "store_name"]

# Handle duplicates (only spacing differences in names, just keep one occurrence of each)
df_stores = df_stores.drop_duplicates(subset = ["store_id"])

# Keep the "Sin Nombre" and "No Identificado" stores, those might be valid stores with no name entered in the DB
# We will see later if those stores are actually active in terms of sells and returns

# Infer several features like total sales in pesos from trains.csv
total_sales_nb = df_train.groupby("store_id")["sales_nb"].sum().to_frame()
df_stores = pd.merge(left = df_stores, right = total_sales_nb, how = "left", left_on = "store_id", right_index = True)
df_stores.rename(columns = {"sales_nb" : "total_sales_nb"}, inplace = True)

total_sales_pesos = df_train.groupby("store_id")["sales_pesos"].sum().to_frame()
df_stores = pd.merge(left = df_stores, right = total_sales_pesos, how = "left", left_on = "store_id", right_index = True)
df_stores.rename(columns = {"sales_pesos" : "total_sales_pesos"}, inplace = True)

total_returns_nb = df_train.groupby("store_id")["returns_nb"].sum().to_frame()
df_stores = pd.merge(left = df_stores, right = total_returns_nb, how = "left", left_on = "store_id", right_index = True)
df_stores.rename(columns = {"returns_nb" : "total_returns_nb"}, inplace = True)

total_returns_pesos = df_train.groupby("store_id")["returns_pesos"].sum().to_frame()
df_stores = pd.merge(left = df_stores, right = total_returns_pesos, how = "left", left_on = "store_id", right_index = True)
df_stores.rename(columns = {"returns_pesos" : "total_returns_pesos"}, inplace = True)

total_adj_demand = df_train.groupby("store_id")["adj_demand"].sum().to_frame()
df_stores = pd.merge(left = df_stores, right = total_adj_demand, how = "left", left_on = "store_id", right_index = True)
df_stores.rename(columns = {"adj_demand" : "total_adj_demand"}, inplace = True)

# Remove rows with NAs
df_stores.dropna(axis = 0, how = "any", inplace = True)

# Use store_id as the index
df_stores.set_index("store_id", inplace = True)

display("Dimensions of Stores table are now : " + str(df_stores.shape))

'Dimensions of Stores table are now : (880604, 6)'

In [5]:
# Now preprocess Depots data
df_depots = pd.read_csv("raw_data/town_state.csv")
df_depots.columns = ["depot_id", "city", "state"]

# Infer several features like total sales in pesos from trains.csv
total_sales_nb = df_train.groupby("depot_id")["sales_nb"].sum().to_frame()
df_depots = pd.merge(left = df_depots, right = total_sales_nb, how = "left", left_on = "depot_id", right_index = True)
df_depots.rename(columns = {"sales_nb" : "total_sales_nb"}, inplace = True)

total_sales_pesos = df_train.groupby("depot_id")["sales_pesos"].sum().to_frame()
df_depots = pd.merge(left = df_depots, right = total_sales_pesos, how = "left", left_on = "depot_id", right_index = True)
df_depots.rename(columns = {"sales_pesos" : "total_sales_pesos"}, inplace = True)

total_returns_nb = df_train.groupby("depot_id")["returns_nb"].sum().to_frame()
df_depots = pd.merge(left = df_depots, right = total_returns_nb, how = "left", left_on = "depot_id", right_index = True)
df_depots.rename(columns = {"returns_nb" : "total_returns_nb"}, inplace = True)

total_returns_pesos = df_train.groupby("depot_id")["returns_pesos"].sum().to_frame()
df_depots = pd.merge(left = df_depots, right = total_returns_pesos, how = "left", left_on = "depot_id", right_index = True)
df_depots.rename(columns = {"returns_pesos" : "total_returns_pesos"}, inplace = True)

total_adj_demand = df_train.groupby("depot_id")["adj_demand"].sum().to_frame()
df_depots = pd.merge(left = df_depots, right = total_adj_demand, how = "left", left_on = "depot_id", right_index = True)
df_depots.rename(columns = {"adj_demand" : "total_adj_demand"}, inplace = True)

# Remove rows with NAs
df_depots.dropna(axis = 0, how = "any", inplace = True)

# Use store_id as the index
df_depots.set_index("depot_id", inplace = True)

display("Dimensions of Depots table are now : " + str(df_depots.shape))

'Dimensions of Depots table are now : (552, 7)'

In [6]:
# Cluster products based on price and weight
temp_products = df_products.drop(["short_name", "product_name", "total_adj_demand", "total_returns_nb", 
                                  "total_sales_nb", "volume", "inches", "pieces", "brand"], axis = 1)
temp_products.dropna(axis = 0, how = "any", inplace = True)
prod_estimator = KMeans(n_clusters = 12)
prod_estimator.fit(temp_products)
temp_products["prod_cluster"] = prod_estimator.labels_
temp_products = temp_products.drop(["avg_price", "weight"], axis = 1)
df_products = pd.merge(left = df_products, right = temp_products, how = "left", left_index = True, right_index = True)
display(df_products.head())

,product_name,short_name,pieces,weight,volume,inches,brand,avg_price,total_sales_nb,total_returns_nb,total_adj_demand,prod_cluster
product_id,,,,,,,,,,,,
41,Bimbollos Ext sAjonjoli 6p 480g BIM 41,Bimbollos Ext sAjonjoli,6.000,480.000,nan,nan,BIM,17.993,22452.000,227.000,22414.000,4.000
53,Burritos Sincro 170g CU LON 53,Burritos Sincro,nan,170.000,nan,nan,LON,14.300,33185.000,1.000,33185.000,0.000
72,Div Tira Mini Doradita 4p 45g TR 72,Div Tira Mini Doradita,4.000,45.000,nan,nan,TR,3.697,722938.000,11795.000,715843.000,0.000
73,Pan Multigrano Linaza 540g BIM 73,Pan Multigrano Linaza,nan,540.000,nan,nan,BIM,21.361,715069.000,17442.000,698935.000,4.000
100,Super Pan Bco Ajonjoli 680g SP WON 100,Super Pan Bco Ajonjoli,nan,680.000,nan,nan,WON,20.388,1166.000,961.000,1093.000,4.000


In [7]:
# Cluster stores based on sales, returns and demand
temp_stores = df_stores.drop("store_name", axis = 1)
temp_stores.dropna(axis = 0, how = "any", inplace = True)
store_estimator = KMeans(n_clusters = 12)
store_estimator.fit(temp_stores)
temp_stores["store_cluster"] = store_estimator.labels_
temp_stores = temp_stores.drop(["total_sales_nb", "total_sales_pesos", "total_returns_nb", "total_returns_pesos", 
                               "total_adj_demand"], axis = 1)
df_stores = pd.merge(left = df_stores, right = temp_stores, how = "left", left_index = True, right_index = True)
display(df_stores.head())

,store_name,total_sales_nb,total_sales_pesos,total_returns_nb,total_returns_pesos,total_adj_demand,store_cluster
store_id,,,,,,,
26,BODEGA COMERCIAL MEXICANA TOLUCA,6845.000,128149.258,531.000,6969.720,6378.000,6
60,SAMS CLUB TOLUCA,46830.000,2148013.500,0.000,0.000,46830.000,8
65,WAL MART METEPEC,68743.000,1113131.500,8.000,197.600,68735.000,4
101,WAL MART TOLUCA,1224.000,27353.881,0.000,0.000,1224.000,7
105,SUPER KOMPRAS SAN BUENAVENTURA,17373.000,310625.156,0.000,0.000,17373.000,2


In [8]:
# Cluster depots based on sales, returns and demand
temp_depots = df_depots.drop(["city", "state"], axis = 1)
temp_depots.dropna(axis = 0, how = "any", inplace = True)
depot_estimator = KMeans(n_clusters = 12)
depot_estimator.fit(temp_depots)
temp_depots["depot_cluster"] = depot_estimator.labels_
temp_depots = temp_depots.drop(["total_sales_nb", "total_sales_pesos", "total_returns_nb", "total_returns_pesos", 
                               "total_adj_demand"], axis = 1)
df_depots = pd.merge(left = df_depots, right = temp_depots, how = "left", left_index = True, right_index = True)
display(df_depots.head())

,city,state,total_sales_nb,total_sales_pesos,total_returns_nb,total_returns_pesos,total_adj_demand,depot_cluster
depot_id,,,,,,,,
1110,2008 AG. LAGO FILT,"MÉXICO, D.F.",877675.000,9274674.000,39900.000,214072.766,874523.000,11
1111,2002 AG. AZCAPOTZALCO,"MÉXICO, D.F.",2720400.000,24070592.000,25231.000,264672.438,2701427.000,1
1112,2004 AG. CUAUTITLAN,ESTADO DE MÉXICO,1959534.000,16591688.000,23924.000,231897.391,1942114.000,4
1113,2008 AG. LAGO FILT,"MÉXICO, D.F.",1442999.000,12094484.000,11865.000,117754.383,1434414.000,2
1114,2029 AG.IZTAPALAPA 2,"MÉXICO, D.F.",3498170.000,62420320.000,150779.000,2480404.750,3363796.000,3


In [9]:
# Add the clustering info in the train DF
temp_products = pd.DataFrame(df_products.prod_cluster)
temp_stores = pd.DataFrame(df_stores.store_cluster)
temp_depots = pd.DataFrame(df_depots.depot_cluster)
df_train = pd.merge(left = df_train, right = temp_products, how = "left", left_on = "product_id", right_index = True)
df_train = pd.merge(left = df_train, right = temp_stores, how = "left", left_on = "store_id", right_index = True)
df_train = pd.merge(left = df_train, right = temp_depots, how = "left", left_on = "depot_id", right_index = True)
display(df_train.head())

,week,depot_id,channel_id,route_id,store_id,product_id,sales_nb,sales_pesos,returns_nb,returns_pesos,adj_demand,product_price,prod_cluster,store_cluster,depot_cluster
0,3,1110,7,3301,15766,1212,3,25.140,0,0.000,3,8.380,0.000,9,11
1,3,1110,7,3301,15766,1216,4,33.520,0,0.000,4,8.380,0.000,9,11
2,3,1110,7,3301,15766,1238,4,39.320,0,0.000,4,9.830,0.000,9,11
3,3,1110,7,3301,15766,1240,4,33.520,0,0.000,4,8.380,0.000,9,11
4,3,1110,7,3301,15766,1242,3,22.920,0,0.000,3,7.640,0.000,9,11


In [10]:
# Load test data
df_test = pd.read_csv("raw_data/test.csv",
                       dtype = {'id' :'int32', 'Semana' : 'int8', 'Agencia_ID' :'int16', 'Canal_ID' : 'int8', 
                                'Ruta_SAK' : 'int16', 'Cliente-ID' : 'int32', 'Producto_ID':'int32'})
df_test.columns = ["id", "week", "depot_id", "channel_id", "route_id", "store_id", "product_id"]

In [11]:
# Predict clustering info for test DF
temp_products = pd.DataFrame(df_products[["weight", "avg_price"]])
temp_products.dropna(axis = 0, how = "any", inplace = True)
temp_products["prod_cluster"] = prod_estimator.predict(temp_products)
temp_products = temp_products.drop(["avg_price", "weight"], axis = 1)
df_test = pd.merge(left = df_test, right = temp_products, how = "left", left_on = "product_id", right_index = True)
###
temp_stores = pd.DataFrame(df_stores[["total_sales_nb", "total_sales_pesos", "total_returns_nb", "total_returns_pesos", 
                                     "total_adj_demand"]])
temp_stores.dropna(axis = 0, how = "any", inplace = True)
temp_stores["store_cluster"] = store_estimator.predict(temp_stores)
temp_stores = temp_stores.drop(["total_sales_nb", "total_sales_pesos", "total_returns_nb", "total_returns_pesos", 
                                "total_adj_demand"], axis = 1)
df_test = pd.merge(left = df_test, right = temp_stores, how = "left", left_on = "store_id", right_index = True)
###
temp_depots = pd.DataFrame(df_depots[["total_sales_nb", "total_sales_pesos", "total_returns_nb", "total_returns_pesos", 
                                     "total_adj_demand"]])
temp_depots.dropna(axis = 0, how = "any", inplace = True)
temp_depots["depot_cluster"] = depot_estimator.predict(temp_depots)
temp_depots = temp_depots.drop(["total_sales_nb", "total_sales_pesos", "total_returns_nb", "total_returns_pesos", 
                                "total_adj_demand"], axis = 1)
df_test = pd.merge(left = df_test, right = temp_depots, how = "left", left_on = "depot_id", right_index = True)

display(df_test.head())

,id,week,depot_id,channel_id,route_id,store_id,product_id,prod_cluster,store_cluster,depot_cluster
0,0,11,4037,1,2209,4639078,35305,0.000,9.000,10
1,1,11,2237,1,1226,4705135,1238,0.000,9.000,1
2,2,10,2045,1,2831,4549769,32940,0.000,9.000,2
3,3,11,1227,1,4448,4717855,43066,0.000,9.000,4
4,4,11,1219,1,1130,966351,1277,0.000,9.000,1


In [13]:
# Save augmented data sets on disk
df_products.to_csv("temp_data/tot_df_products.csv", index = False, encoding = "utf-8")
df_stores.to_csv("temp_data/tot_df_stores.csv", index = False, encoding = "utf-8")
df_depots.to_csv("temp_data/tot_df_depots.csv", index = False, encoding = "utf-8")
df_train.to_csv("temp_data/tot_df_train.csv", index = False, encoding = "utf-8")
df_test.to_csv("temp_data/tot_df_test.csv", index = False, encoding = "utf-8")